# Advanced Feature Engineering for Credit Default Prediction
## Goal: Improve AUC from 0.7889 to > 0.80

### Strategy:
1. Interaction features from identified candidates
2. Polynomial features for top predictors
3. Binning for high-skew features
4. Sparse feature indicators
5. Domain-specific credit risk features
6. Ratio and aggregation features

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import KBinsDiscretizer
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

print("Loading data...")

In [ ]:
# Load datasets
X_train = pd.read_parquet('/home/dr/cbu/X_train.parquet')
X_test = pd.read_parquet('/home/dr/cbu/X_test.parquet')
y_train = pd.read_parquet('/home/dr/cbu/y_train.parquet').values.ravel()

print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")
print(f"Target: {len(y_train)} samples, {y_train.sum()} defaults ({y_train.mean():.2%})")

original_features = X_train.columns.tolist()
original_count = len(original_features)

In [ ]:
# Load analysis results
numeric_stats = pd.read_csv('/home/dr/cbu/numeric_features_statistics.csv')
class_separation = pd.read_csv('/home/dr/cbu/class_separation_analysis.csv')
interactions = pd.read_csv('/home/dr/cbu/interaction_candidates.csv')

print("Analysis files loaded successfully")
print(f"\nTop 10 features by KS-statistic:")
print(class_separation.nlargest(10, 'ks_statistic')[['feature', 'ks_statistic', 'median_difference_pct']])

## 1. Interaction Features

In [ ]:
print("Creating interaction features...")

# Top interaction candidates based on correlation analysis
interaction_pairs = [
    ('debt_service_ratio', 'payment_to_income_ratio'),
    ('debt_service_ratio', 'payment_burden'),
    ('payment_to_income_ratio', 'total_debt_to_income'),
    ('annual_income', 'age'),
    ('credit_score', 'debt_to_income_ratio'),
    ('credit_score', 'annual_income'),
    ('income_vs_regional', 'debt_service_ratio'),
    ('age', 'employment_length'),
    ('num_delinquencies_2yrs', 'credit_score'),
    ('credit_utilization', 'revolving_balance'),
    ('debt_to_income_ratio', 'credit_utilization'),
    ('monthly_income', 'existing_monthly_debt'),
    ('total_debt_amount', 'annual_income'),
    ('available_credit', 'credit_utilization')
]

interaction_count = 0
for feat1, feat2 in interaction_pairs:
    if feat1 in X_train.columns and feat2 in X_train.columns:
        # Multiplicative interaction
        interaction_name = f"{feat1}_X_{feat2}"
        X_train[interaction_name] = X_train[feat1] * X_train[feat2]
        X_test[interaction_name] = X_test[feat1] * X_test[feat2]
        interaction_count += 1
        
        # Ratio interaction (safe division)
        if not (X_train[feat2] == 0).any():
            ratio_name = f"{feat1}_div_{feat2}"
            X_train[ratio_name] = X_train[feat1] / (X_train[feat2] + 1e-10)
            X_test[ratio_name] = X_test[feat1] / (X_test[feat2] + 1e-10)
            interaction_count += 1

print(f"Created {interaction_count} interaction features")

## 2. Polynomial Features for Top Predictors

In [ ]:
print("Creating polynomial features...")

# Top 5 predictors
top_features = class_separation.nlargest(5, 'ks_statistic')['feature'].tolist()
top_features = [f for f in top_features if f in X_train.columns]

poly_count = 0
for feature in top_features:
    # Square term
    X_train[f"{feature}_squared"] = X_train[feature] ** 2
    X_test[f"{feature}_squared"] = X_test[feature] ** 2
    poly_count += 1
    
    # Cube term (only for features with reasonable range)
    if X_train[feature].abs().max() < 100:
        X_train[f"{feature}_cubed"] = X_train[feature] ** 3
        X_test[f"{feature}_cubed"] = X_test[feature] ** 3
        poly_count += 1
    
    # Square root (for non-negative features)
    if (X_train[feature] >= 0).all():
        X_train[f"{feature}_sqrt"] = np.sqrt(X_train[feature])
        X_test[f"{feature}_sqrt"] = np.sqrt(X_test[feature])
        poly_count += 1
    
    # Log transformation (for positive features)
    if (X_train[feature] > 0).all():
        X_train[f"{feature}_log"] = np.log1p(X_train[feature])
        X_test[f"{feature}_log"] = np.log1p(X_test[feature])
        poly_count += 1

print(f"Created {poly_count} polynomial features for {len(top_features)} top predictors")

## 3. Binned Features for High-Skew Variables

In [ ]:
print("Creating binned features...")

# High skew features
high_skew = numeric_stats[numeric_stats['skewness'].abs() > 3]['feature'].tolist()
high_skew = [f for f in high_skew if f in X_train.columns][:10]

binned_count = 0
for feature in high_skew:
    try:
        kbd = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
        
        binned_train = kbd.fit_transform(X_train[[feature]])
        binned_test = kbd.transform(X_test[[feature]])
        
        X_train[f"{feature}_binned"] = binned_train
        X_test[f"{feature}_binned"] = binned_test
        binned_count += 1
    except:
        continue

print(f"Created {binned_count} binned features")

## 4. Sparse Feature Indicators

In [ ]:
print("Creating sparse feature indicators...")

# Sparse features (>50% zeros)
sparse_features = numeric_stats[numeric_stats['zero_percentage'] > 50]['feature'].tolist()
sparse_features = [f for f in sparse_features if f in X_train.columns][:20]

sparse_count = 0
for feature in sparse_features:
    # Binary indicator: has non-zero value
    X_train[f"{feature}_has_value"] = (X_train[feature] != 0).astype(int)
    X_test[f"{feature}_has_value"] = (X_test[feature] != 0).astype(int)
    sparse_count += 1
    
    # For very sparse features, create magnitude indicator
    zero_pct = numeric_stats[numeric_stats['feature'] == feature]['zero_percentage'].values[0]
    if zero_pct > 80:
        non_zero = X_train[feature][X_train[feature] != 0]
        if len(non_zero) > 10:
            q1, q3 = non_zero.quantile([0.25, 0.75])
            X_train[f"{feature}_magnitude"] = pd.cut(
                X_train[feature],
                bins=[-np.inf, 0, q1, q3, np.inf],
                labels=[0, 1, 2, 3]
            ).astype(float)
            X_test[f"{feature}_magnitude"] = pd.cut(
                X_test[feature],
                bins=[-np.inf, 0, q1, q3, np.inf],
                labels=[0, 1, 2, 3]
            ).astype(float)
            sparse_count += 1

print(f"Created {sparse_count} sparse feature indicators")

## 5. Domain-Specific Credit Risk Features

In [ ]:
print("Creating domain-specific features...")

domain_count = 0

# 1. Debt burden score
if all(f in X_train.columns for f in ['annual_income', 'total_debt_amount', 'credit_score']):
    X_train['debt_burden_score'] = (
        X_train['total_debt_amount'] / (X_train['annual_income'] + 1) *
        (1000 - X_train['credit_score']) / 1000
    )
    X_test['debt_burden_score'] = (
        X_test['total_debt_amount'] / (X_test['annual_income'] + 1) *
        (1000 - X_test['credit_score']) / 1000
    )
    domain_count += 1

# 2. Credit utilization categories
if 'credit_utilization' in X_train.columns:
    X_train['credit_util_category'] = pd.cut(
        X_train['credit_utilization'],
        bins=[0, 0.3, 0.5, 0.7, 1.0],
        labels=[1, 2, 3, 4]
    ).astype(float).fillna(0)
    X_test['credit_util_category'] = pd.cut(
        X_test['credit_utilization'],
        bins=[0, 0.3, 0.5, 0.7, 1.0],
        labels=[1, 2, 3, 4]
    ).astype(float).fillna(0)
    domain_count += 1

# 3. Income vs expected by age
if all(f in X_train.columns for f in ['age', 'annual_income']):
    expected_income_train = 20000 + (X_train['age'] - 18) * 2000
    expected_income_test = 20000 + (X_test['age'] - 18) * 2000
    
    X_train['income_vs_age_expected'] = X_train['annual_income'] / expected_income_train
    X_test['income_vs_age_expected'] = X_test['annual_income'] / expected_income_test
    domain_count += 1

# 4. Payment capacity
if all(f in X_train.columns for f in ['monthly_free_cash_flow', 'monthly_payment']):
    X_train['payment_capacity'] = X_train['monthly_free_cash_flow'] / (X_train['monthly_payment'] + 100)
    X_test['payment_capacity'] = X_test['monthly_free_cash_flow'] / (X_test['monthly_payment'] + 100)
    domain_count += 1

# 5. Combined risk score
risk_factors = ['num_delinquencies_2yrs', 'debt_service_ratio', 'payment_to_income_ratio']
available_risk = [f for f in risk_factors if f in X_train.columns]

if available_risk:
    risk_scores_train = pd.DataFrame()
    risk_scores_test = pd.DataFrame()
    
    for factor in available_risk:
        min_val = X_train[factor].min()
        max_val = X_train[factor].max()
        risk_scores_train[factor] = (X_train[factor] - min_val) / (max_val - min_val + 1e-10)
        risk_scores_test[factor] = (X_test[factor] - min_val) / (max_val - min_val + 1e-10)
    
    X_train['combined_risk_score'] = risk_scores_train.mean(axis=1)
    X_test['combined_risk_score'] = risk_scores_test.mean(axis=1)
    domain_count += 1

# 6. Financial stress indicator
if all(f in X_train.columns for f in ['debt_to_income_ratio', 'credit_utilization']):
    X_train['financial_stress'] = X_train['debt_to_income_ratio'] * X_train['credit_utilization']
    X_test['financial_stress'] = X_test['debt_to_income_ratio'] * X_test['credit_utilization']
    domain_count += 1

# 7. Credit score risk bucket
if 'credit_score' in X_train.columns:
    X_train['credit_score_bucket'] = pd.cut(
        X_train['credit_score'],
        bins=[0, 580, 670, 740, 800, 1000],
        labels=[5, 4, 3, 2, 1]  # Higher number = higher risk
    ).astype(float).fillna(5)
    X_test['credit_score_bucket'] = pd.cut(
        X_test['credit_score'],
        bins=[0, 580, 670, 740, 800, 1000],
        labels=[5, 4, 3, 2, 1]
    ).astype(float).fillna(5)
    domain_count += 1

print(f"Created {domain_count} domain-specific features")

## 6. Additional Ratio Features

In [ ]:
print("Creating additional ratio features...")

ratio_pairs = [
    ('revolving_balance', 'available_credit'),
    ('monthly_payment', 'monthly_income'),
    ('existing_monthly_debt', 'monthly_income'),
    ('total_debt_amount', 'annual_income'),
    ('num_customer_service_calls', 'account_tenure_years'),
    ('num_login_sessions', 'account_tenure_years')
]

ratio_count = 0
for numerator, denominator in ratio_pairs:
    if numerator in X_train.columns and denominator in X_train.columns:
        ratio_name = f"ratio_{numerator}_to_{denominator}"
        X_train[ratio_name] = X_train[numerator] / (X_train[denominator] + 1e-10)
        X_test[ratio_name] = X_test[numerator] / (X_test[denominator] + 1e-10)
        
        # Cap extreme values
        cap_value = X_train[ratio_name].quantile(0.99)
        X_train[ratio_name] = X_train[ratio_name].clip(upper=cap_value)
        X_test[ratio_name] = X_test[ratio_name].clip(upper=cap_value)
        ratio_count += 1

print(f"Created {ratio_count} additional ratio features")

## 7. Aggregation Features

In [ ]:
print("Creating aggregation features...")

agg_count = 0

# Debt-related features
debt_features = [f for f in X_train.columns if 'debt' in f.lower() and f in original_features]
if len(debt_features) > 1:
    X_train['debt_features_mean'] = X_train[debt_features].mean(axis=1)
    X_train['debt_features_std'] = X_train[debt_features].std(axis=1)
    X_train['debt_features_max'] = X_train[debt_features].max(axis=1)
    
    X_test['debt_features_mean'] = X_test[debt_features].mean(axis=1)
    X_test['debt_features_std'] = X_test[debt_features].std(axis=1)
    X_test['debt_features_max'] = X_test[debt_features].max(axis=1)
    agg_count += 3

# Payment-related features
payment_features = [f for f in X_train.columns if 'payment' in f.lower() and f in original_features]
if len(payment_features) > 1:
    X_train['payment_features_mean'] = X_train[payment_features].mean(axis=1)
    X_train['payment_features_sum'] = X_train[payment_features].sum(axis=1)
    
    X_test['payment_features_mean'] = X_test[payment_features].mean(axis=1)
    X_test['payment_features_sum'] = X_test[payment_features].sum(axis=1)
    agg_count += 2

print(f"Created {agg_count} aggregation features")

## 8. Validation and Cleanup

In [ ]:
print("\nValidating features...")

# Replace infinite values
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

# Fill NaN values
nan_cols_train = X_train.columns[X_train.isna().any()].tolist()
nan_cols_test = X_test.columns[X_test.isna().any()].tolist()

if nan_cols_train or nan_cols_test:
    print(f"Filling NaN values in {len(set(nan_cols_train + nan_cols_test))} features")
    for col in set(nan_cols_train + nan_cols_test):
        if col in X_train.columns:
            median_val = X_train[col].median()
            X_train[col] = X_train[col].fillna(median_val)
            X_test[col] = X_test[col].fillna(median_val)

# Remove low variance features
low_var_features = numeric_stats[numeric_stats['std'] < 0.01]['feature'].tolist()
low_var_to_remove = [f for f in low_var_features if f in X_train.columns]

if low_var_to_remove:
    X_train = X_train.drop(columns=low_var_to_remove)
    X_test = X_test.drop(columns=low_var_to_remove)
    print(f"Removed {len(low_var_to_remove)} low-variance features")

print(f"\nFinal shape - Train: {X_train.shape}, Test: {X_test.shape}")
print(f"New features created: {X_train.shape[1] - original_count}")

## 9. Feature Quality Assessment

In [ ]:
print("\nAssessing feature quality...")

# Calculate correlations with target for new features
new_features = [f for f in X_train.columns if f not in original_features]
correlations = {}

for col in new_features[:100]:  # Check first 100 new features
    corr = np.abs(np.corrcoef(X_train[col], y_train)[0, 1])
    if not np.isnan(corr):
        correlations[col] = corr

# Sort by correlation
top_new_features = sorted(correlations.items(), key=lambda x: x[1], reverse=True)[:20]

print("\nTop 20 new features by target correlation:")
for i, (feat, corr) in enumerate(top_new_features, 1):
    print(f"{i:2d}. {feat[:60]:60s} {corr:.4f}")

# Save correlation results
corr_df = pd.DataFrame(list(correlations.items()), columns=['feature', 'abs_correlation'])
corr_df = corr_df.sort_values('abs_correlation', ascending=False)
corr_df.to_csv('/home/dr/cbu/new_features_correlations.csv', index=False)
print("\nSaved correlation analysis to new_features_correlations.csv")

## 10. Save Engineered Features

In [ ]:
print("\nSaving engineered features...")

X_train.to_parquet('/home/dr/cbu/X_train_engineered.parquet', engine='pyarrow', compression='snappy')
X_test.to_parquet('/home/dr/cbu/X_test_engineered.parquet', engine='pyarrow', compression='snappy')

print(f"Saved X_train_engineered.parquet: {X_train.shape}")
print(f"Saved X_test_engineered.parquet: {X_test.shape}")
print("\nFeature engineering completed successfully!")

## Summary Report

In [ ]:
print("\n" + "="*80)
print("FEATURE ENGINEERING SUMMARY")
print("="*80)
print(f"\nOriginal features: {original_count}")
print(f"Final features: {X_train.shape[1]}")
print(f"New features: {X_train.shape[1] - original_count}")
print(f"\nTraining samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"\nDefault rate: {y_train.mean():.2%}")
print(f"Class imbalance ratio: 1:{int(1/y_train.mean())}")
print("\n" + "="*80)